In [1]:
import pandas as pd
import os
import numpy as np
import librosa
from sklearn.preprocessing import OneHotEncoder

In [2]:
paths=[]
labels=[]
for filename in os.listdir('./AudioWAV'):
    
    paths.append('./AudioWAV/' + filename)
    file = filename.split('.')[0]
   
    label = file.split('_')[2]
    if label == 'ANG':
        labels.append('angry.wav')
    elif label == 'DIS':
        labels.append('disgust.wav')
    elif label == 'FEA':
        labels.append('fear.wav')
    elif label == 'HAP':
        labels.append('happy.wav')
    elif label == 'NEU':
        labels.append('neutral.wav')
    elif label == 'SAD':
        labels.append('sad.wav')
        

df_cremad = pd.DataFrame({'speech':paths,'label':labels})
df_cremad.sample(5)

,speech,label
4795,./AudioWAV/1059_TAI_ANG_XX.wav,angry.wav
5985,./AudioWAV/1074_IEO_NEU_XX.wav,neutral.wav
279,./AudioWAV/1004_ITS_ANG_XX.wav,angry.wav
3741,./AudioWAV/1046_TSI_ANG_XX.wav,angry.wav
219,./AudioWAV/1003_MTI_NEU_XX.wav,neutral.wav


In [3]:
def MFCC(filename):
    y, sr = librosa.load(filename,duration=3,offset=0.5)
    return np.mean(librosa.feature.mfcc(y=y,sr=sr,n_mfcc=40).T,axis=0)

mfcc_cremad = df_cremad['speech'].apply(lambda x:MFCC(x))

In [4]:
X =[x for x in mfcc_cremad]
X =np.array(X)
X.shape
X =np.expand_dims(X,-1)
X.shape

(7442, 40, 1)

In [5]:
ohe=OneHotEncoder()
y = ohe.fit_transform(df_cremad[['label']] )
y = y.toarray()

In [6]:
X.shape, y.shape

((7442, 40, 1), (7442, 6))

In [7]:
df_cremad['label'].unique()

array(['angry.wav', 'disgust.wav', 'fear.wav', 'happy.wav', 'neutral.wav',
       'sad.wav'], dtype=object)

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Definindo os modelos

class SimpleDNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleDNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

class LSTMAttention(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.attention = nn.Linear(hidden_size, 1)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        attn_weights = torch.softmax(self.attention(out), dim=1)
        out = torch.sum(attn_weights * out, dim=1)
        out = self.fc(out)
        return out

class CNNModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 32, kernel_size=3, padding=1)
        self.fc_input_size = 32 * 1 * 1
        self.fc1 = nn.Linear(self.fc_input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.dropout(x)
        x = x.view(-1, self.fc_input_size)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


class CNNAttention(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNNAttention, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 32, kernel_size=3, padding=1)
        self.attention = nn.Linear(32, 1)
        self.fc = nn.Linear(32, num_classes)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        attn_weights = self.softmax(self.attention(x.permute(0, 2, 1))).squeeze(-1)
        attn_weights = attn_weights.unsqueeze(-1)
        x = torch.sum(attn_weights * x, dim=2)
        x = self.fc(x)
        return x
    
# Construindo e treinando os modelos

input_size = X.shape[1:]
num_classes = y.shape[1]
hidden_size = 64
num_layers = 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convertendo os dados para tensores PyTorch
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).to(device)

# Definindo o tamanho do lote
batch_size = 32

# Criando conjuntos de dados PyTorch
dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Definindo tensor para o LSTM
X_tensorLSTM = X_tensor.permute(0, 2, 1)

# Criando conjuntos de dados PyTorch para LSTM
datasetLSTM = torch.utils.data.TensorDataset(X_tensorLSTM, y_tensor)
train_sizeLSTM = int(0.8 * len(datasetLSTM))
test_sizeLSTM = len(datasetLSTM) - train_sizeLSTM
train_datasetLSTM, test_datasetLSTM = torch.utils.data.random_split(datasetLSTM, [train_sizeLSTM, test_sizeLSTM])

# DataLoader para o LSTM
train_loaderLSTM = DataLoader(train_datasetLSTM, batch_size=batch_size, shuffle=True)
test_loaderLSTM = DataLoader(test_datasetLSTM, batch_size=batch_size, shuffle=False)

# Convertendo os dados para tensores PyTorch do CNN
X_tensorCNN = torch.tensor(X, dtype=torch.float32).to(device)

# Criando conjuntos de dados PyTorch do CNN
datasetCNN = torch.utils.data.TensorDataset(X_tensorCNN, y_tensor)
train_sizeCNN = int(0.8 * len(datasetCNN))
test_sizeCNN = len(datasetCNN) - train_sizeCNN
train_datasetCNN, test_datasetCNN = torch.utils.data.random_split(datasetCNN, [train_sizeCNN, test_sizeCNN])

# DataLoader para o CNN
train_loaderCNN = DataLoader(train_datasetCNN, batch_size=batch_size, shuffle=True)
test_loaderCNN = DataLoader(test_datasetCNN, batch_size=batch_size, shuffle=False)

# Função para treinamento
def train(model, train_loader, criterion, optimizer, num_epochs=200):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            #print(outputs.shape)
            loss = criterion(outputs, torch.max(labels, 1)[1])
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == torch.max(labels, 1)[1]).sum().item()

        epoch_loss = running_loss / len(train_loader)
        accuracy = correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}')

In [9]:
# Standard Deep Neural Network
sdnn_model = SimpleDNN(input_size[0], num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(sdnn_model.parameters(), lr=0.001)
train(sdnn_model, train_loader, criterion, optimizer)

Epoch [1/200], Loss: 2.8810, Accuracy: 0.1776
Epoch [2/200], Loss: 1.8046, Accuracy: 0.1757
Epoch [3/200], Loss: 1.7951, Accuracy: 0.1833
Epoch [4/200], Loss: 1.7724, Accuracy: 0.1970
Epoch [5/200], Loss: 1.7304, Accuracy: 0.2261
Epoch [6/200], Loss: 1.6900, Accuracy: 0.2501
Epoch [7/200], Loss: 1.6880, Accuracy: 0.2530
Epoch [8/200], Loss: 1.6693, Accuracy: 0.2681
Epoch [9/200], Loss: 1.6616, Accuracy: 0.2911
Epoch [10/200], Loss: 1.6404, Accuracy: 0.3047
Epoch [11/200], Loss: 1.6376, Accuracy: 0.2948
Epoch [12/200], Loss: 1.6146, Accuracy: 0.3010
Epoch [13/200], Loss: 1.6175, Accuracy: 0.3067
Epoch [14/200], Loss: 1.6093, Accuracy: 0.3072
Epoch [15/200], Loss: 1.5924, Accuracy: 0.3208
Epoch [16/200], Loss: 1.5913, Accuracy: 0.3163
Epoch [17/200], Loss: 1.5835, Accuracy: 0.3229
Epoch [18/200], Loss: 1.5723, Accuracy: 0.3303
Epoch [19/200], Loss: 1.5711, Accuracy: 0.3329
Epoch [20/200], Loss: 1.5815, Accuracy: 0.3190
Epoch [21/200], Loss: 1.5716, Accuracy: 0.3271
Epoch [22/200], Loss: 

In [10]:
# LSTM
lstm_model = LSTMModel(input_size[0], hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)
train(lstm_model, train_loaderLSTM, criterion, optimizer)

Epoch [1/200], Loss: 1.6698, Accuracy: 0.2894
Epoch [2/200], Loss: 1.5579, Accuracy: 0.3388
Epoch [3/200], Loss: 1.5366, Accuracy: 0.3477
Epoch [4/200], Loss: 1.5211, Accuracy: 0.3670
Epoch [5/200], Loss: 1.5068, Accuracy: 0.3684
Epoch [6/200], Loss: 1.4958, Accuracy: 0.3751
Epoch [7/200], Loss: 1.4861, Accuracy: 0.3845
Epoch [8/200], Loss: 1.4710, Accuracy: 0.3931
Epoch [9/200], Loss: 1.4665, Accuracy: 0.3902
Epoch [10/200], Loss: 1.4707, Accuracy: 0.3964
Epoch [11/200], Loss: 1.4391, Accuracy: 0.4052
Epoch [12/200], Loss: 1.4307, Accuracy: 0.4111
Epoch [13/200], Loss: 1.4188, Accuracy: 0.4211
Epoch [14/200], Loss: 1.4187, Accuracy: 0.4136
Epoch [15/200], Loss: 1.4064, Accuracy: 0.4208
Epoch [16/200], Loss: 1.3895, Accuracy: 0.4284
Epoch [17/200], Loss: 1.3807, Accuracy: 0.4272
Epoch [18/200], Loss: 1.3846, Accuracy: 0.4373
Epoch [19/200], Loss: 1.3750, Accuracy: 0.4391
Epoch [20/200], Loss: 1.3606, Accuracy: 0.4430
Epoch [21/200], Loss: 1.3525, Accuracy: 0.4448
Epoch [22/200], Loss: 

In [11]:
# LSTM with Attention
lstm_atn_model = LSTMAttention(input_size[0], hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_atn_model.parameters(), lr=0.001)
train(lstm_atn_model, train_loaderLSTM, criterion, optimizer)

Epoch [1/200], Loss: 1.6588, Accuracy: 0.2950
Epoch [2/200], Loss: 1.5541, Accuracy: 0.3496
Epoch [3/200], Loss: 1.5363, Accuracy: 0.3610
Epoch [4/200], Loss: 1.5121, Accuracy: 0.3726
Epoch [5/200], Loss: 1.4958, Accuracy: 0.3758
Epoch [6/200], Loss: 1.4707, Accuracy: 0.3973
Epoch [7/200], Loss: 1.4537, Accuracy: 0.4000
Epoch [8/200], Loss: 1.4326, Accuracy: 0.4047
Epoch [9/200], Loss: 1.4178, Accuracy: 0.4151
Epoch [10/200], Loss: 1.4195, Accuracy: 0.4191
Epoch [11/200], Loss: 1.4030, Accuracy: 0.4226
Epoch [12/200], Loss: 1.3971, Accuracy: 0.4347
Epoch [13/200], Loss: 1.3985, Accuracy: 0.4263
Epoch [14/200], Loss: 1.3902, Accuracy: 0.4295
Epoch [15/200], Loss: 1.3702, Accuracy: 0.4376
Epoch [16/200], Loss: 1.3644, Accuracy: 0.4497
Epoch [17/200], Loss: 1.3698, Accuracy: 0.4418
Epoch [18/200], Loss: 1.3637, Accuracy: 0.4441
Epoch [19/200], Loss: 1.3506, Accuracy: 0.4460
Epoch [20/200], Loss: 1.3495, Accuracy: 0.4448
Epoch [21/200], Loss: 1.3547, Accuracy: 0.4527
Epoch [22/200], Loss: 

In [12]:
# Convolutional Neural Network
cnn_model = CNNModel(input_size[0],num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)
train(cnn_model, train_loaderCNN, criterion, optimizer)

Epoch [1/200], Loss: 1.7136, Accuracy: 0.2466
Epoch [2/200], Loss: 1.6368, Accuracy: 0.2886
Epoch [3/200], Loss: 1.6027, Accuracy: 0.3183
Epoch [4/200], Loss: 1.5774, Accuracy: 0.3178
Epoch [5/200], Loss: 1.5693, Accuracy: 0.3266
Epoch [6/200], Loss: 1.5528, Accuracy: 0.3370
Epoch [7/200], Loss: 1.5429, Accuracy: 0.3430
Epoch [8/200], Loss: 1.5430, Accuracy: 0.3375
Epoch [9/200], Loss: 1.5307, Accuracy: 0.3425
Epoch [10/200], Loss: 1.5228, Accuracy: 0.3465
Epoch [11/200], Loss: 1.5088, Accuracy: 0.3581
Epoch [12/200], Loss: 1.5125, Accuracy: 0.3555
Epoch [13/200], Loss: 1.4992, Accuracy: 0.3585
Epoch [14/200], Loss: 1.4991, Accuracy: 0.3670
Epoch [15/200], Loss: 1.4940, Accuracy: 0.3632
Epoch [16/200], Loss: 1.4888, Accuracy: 0.3660
Epoch [17/200], Loss: 1.4817, Accuracy: 0.3724
Epoch [18/200], Loss: 1.4830, Accuracy: 0.3741
Epoch [19/200], Loss: 1.4649, Accuracy: 0.3766
Epoch [20/200], Loss: 1.4689, Accuracy: 0.3822
Epoch [21/200], Loss: 1.4617, Accuracy: 0.3828
Epoch [22/200], Loss: 

In [13]:
# Convolutional Neural Network with Attention
cnn_atn_model = CNNAttention(input_size[0],num_classes).to(device)
optimizer = optim.Adam(cnn_atn_model.parameters(), lr=0.001)
train(cnn_atn_model, train_loaderCNN, criterion, optimizer)

Epoch [1/200], Loss: 1.7601, Accuracy: 0.2703
Epoch [2/200], Loss: 1.5680, Accuracy: 0.3387
Epoch [3/200], Loss: 1.5330, Accuracy: 0.3578
Epoch [4/200], Loss: 1.5090, Accuracy: 0.3677
Epoch [5/200], Loss: 1.4774, Accuracy: 0.3837
Epoch [6/200], Loss: 1.4736, Accuracy: 0.3968
Epoch [7/200], Loss: 1.4563, Accuracy: 0.4027
Epoch [8/200], Loss: 1.4333, Accuracy: 0.4111
Epoch [9/200], Loss: 1.4334, Accuracy: 0.4099
Epoch [10/200], Loss: 1.4410, Accuracy: 0.4060
Epoch [11/200], Loss: 1.4092, Accuracy: 0.4189
Epoch [12/200], Loss: 1.4149, Accuracy: 0.4176
Epoch [13/200], Loss: 1.4037, Accuracy: 0.4242
Epoch [14/200], Loss: 1.3975, Accuracy: 0.4258
Epoch [15/200], Loss: 1.4002, Accuracy: 0.4287
Epoch [16/200], Loss: 1.3883, Accuracy: 0.4319
Epoch [17/200], Loss: 1.3626, Accuracy: 0.4420
Epoch [18/200], Loss: 1.3644, Accuracy: 0.4406
Epoch [19/200], Loss: 1.3606, Accuracy: 0.4465
Epoch [20/200], Loss: 1.3532, Accuracy: 0.4425
Epoch [21/200], Loss: 1.3344, Accuracy: 0.4546
Epoch [22/200], Loss: 

In [14]:
# Função para teste
def test(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, torch.max(labels, 1)[1])
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == torch.max(labels, 1)[1]).sum().item()

    test_loss = running_loss / len(test_loader)
    test_accuracy = correct / total
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

In [15]:
print("Testing SimpleDNN:")
test(sdnn_model, test_loader, criterion)

Testing SimpleDNN:
Test Loss: 1.4349, Test Accuracy: 0.4291


In [16]:
print("\nTesting LSTMModel:")
test(lstm_model, test_loaderLSTM, criterion)



Testing LSTMModel:


Test Loss: 1.6032, Test Accuracy: 0.4305


In [17]:
print("\nTesting LSTMAttention:")
test(lstm_atn_model, test_loaderLSTM, criterion)


Testing LSTMAttention:
Test Loss: 1.7107, Test Accuracy: 0.4338


In [18]:
print("\nTesting CNNModel:")
test(cnn_model, test_loaderCNN, criterion)



Testing CNNModel:
Test Loss: 1.4820, Test Accuracy: 0.3875


In [19]:
print("\nTesting CNNAttention:")
test(cnn_atn_model, test_loaderCNN, criterion)


Testing CNNAttention:
Test Loss: 1.8222, Test Accuracy: 0.4090
